In [2]:
import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe
# tf.compat.v1.enable_eager_execution()
print(tf.__version__)

import time
import ingestor
import extractor2
import utils
import data
import functools

1.14.0
Tesor Flow Version: 1.14.0 Ingest File
Tesor Flow Version: 1.14.0  Utility File
Tesor Flow Version: 1.14.0 Extactor 2


In [3]:
params = ingestor.get_params()

In [4]:
train_data = data.get_train_dataset(params)
parser = data.tristan_parser
train_parsed = train_data.map(functools.partial(parser, params=params))

In [81]:
# test for checking what train_parsed contains
features = {'goal': [], 'goal_asl': [], 'thms': [], 'thms_hard_negatives': []}
labels = {'tac_id': []}

#remove 'take' part to iterate over the entire dataset
for raw_record in train_parsed.take(2):
    fx, lx = raw_record[0], raw_record[1]
    features['goal'].append(fx['goal'])
    features['goal_asl'].append(fx['goal_asl'])
    features['thms'].append(fx['thms'])
    features['thms_hard_negatives'].append(fx['thms_hard_negatives'])
    labels['tac_id'].append(lx['tac_id'])

# I went through and checked how tokenize works and it makes sense to me
# I think we should join the goal and thms vocab
ex = extractor2.Extractor(params)
features['goal_ids'] = ex.tokenize(features['goal'], ex.goal_table)
# We need to make this an iterator over length of the dataset
temp1 = ex.tokenize(features['thms'][0], ex.thms_table)
temp2 = ex.tokenize(features['thms'][1], ex.thms_table)
features['thms_ids'] = [temp1, temp2]
# Need to do the same for goal_asl - use joint vocab
# Should we do one-hot encoding?

In [82]:
# # # test for checking what train_parsed contains
# for raw_record in train_parsed.take(1):
#   print(repr(raw_record))